## 【ゼロ過剰ポアソン分布】出張回数

出張回数のデータをゼロ過剰ポアソン分布にあてはめて、情報量基準（WAIC）により比較する。

In [ ]:
import pymc as pm
import arviz as az

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

## Load & Check Data

In [ ]:
data = pd.read_csv('data.csv')

In [ ]:
data

In [ ]:
x = data['count'].values

print(x)

## Define Model & Inference

In [ ]:
with pm.Model() as model:

    psi = pm.Uniform('psi', lower=0, upper=1)
    lam = pm.HalfNormal('lam', sigma=10)

    obs = pm.ZeroInflatedPoisson('obs', psi=psi, mu=lam, observed=x)

In [ ]:
with model:

    trace = pm.sample(3000, return_inferencedata=False)
    idata = pm.to_inference_data(trace, log_likelihood=True)

In [ ]:
idata

## Check MCMC-samples

In [ ]:
az.plot_trace(idata)
plt.gcf().subplots_adjust(wspace=0.5, hspace=0.5)

In [ ]:
az.summary(idata)

In [ ]:
az.plot_posterior(idata);

## Posterior Predictive Check

In [ ]:
with model:

    idata_ppc = pm.sample_posterior_predictive(idata)

In [ ]:
az.plot_ppc(idata_ppc, num_pp_samples=1000);

In [ ]:
az.plot_ppc(idata_ppc, kind='cumulative', num_pp_samples=1000);

## Information Criteria

In [ ]:
az.waic(idata, scale='deviance')

In [ ]:
az.loo(idata, scale='deviance')

## Compare Models

In [ ]:
idata_poisson = az.from_netcdf('idata.nc')

In [ ]:
dict_idata = {'Poisson':idata_poisson, 'Zero Inflated Poisson':idata}

In [ ]:
df_waic = az.compare(dict_idata, ic='waic', scale='deviance')
df_waic

In [ ]:
az.plot_compare(df_waic, figsize=(8, 3));

In [ ]:
df_loo = az.compare(dict_idata, ic='loo', scale='deviance')
df_loo

In [ ]:
az.plot_compare(df_loo, figsize=(8, 3));